In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm  
from PIL import Image
import torch
from health_multimodal.vlp.inference_engine import ImageTextInferenceEngine
from health_multimodal.image.data.transforms import create_chest_xray_transform_for_inference
from health_multimodal.image.model.pretrained import get_biovil_t_image_encoder
from health_multimodal.text.utils import BertEncoderType, get_bert_inference
from health_multimodal.image import ImageInferenceEngine

/Users/solkim/Desktop/projects/medical/project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/solkim/Desktop/projects/medical/project/.venv/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [31]:
# ✅ [최종-진짜] get_image_embedding 함수가 model과 transform을 직접 받도록 수정합니다.
def get_image_embedding(image_path: str, model, transform) -> np.ndarray:
    """주어진 이미지 경로에 대해 임베딩 벡터를 추출하는 최종 헬퍼 함수"""
    image_path_obj = Path(image_path)
    if not image_path_obj.exists():
        return None
    
    # ✅ [최종-완결] 이미지를 1-채널 흑백('L')으로 강제 변환하여 모든 이미지 형식을 통일합니다.
    pil_image = Image.open(image_path_obj).convert('L')
    image_tensor = transform(pil_image)
    batch_tensor = image_tensor.unsqueeze(0).to(device)
    
    with torch.no_grad():
        model_output = model(batch_tensor)
    
    # ✅ [최종 수정] 모델 출력 객체에서 .img_embedding 속성을 추출합니다.
    embedding_tensor = model_output.img_embedding
    
    return embedding_tensor.detach().numpy()

In [32]:
# --- 1. 사용자 설정 부분 ---
csv_path = '/Users/solkim/Desktop/projects/medical/project/data/chestxray14/BBox_List_2017.csv'
image_dir = '/Users/solkim/Desktop/projects/medical/project/data/chestxray14/bbox_images'
ACTUAL_IMAGE_COLUMN_NAME = 'Image Index'
ACTUAL_LABEL_COLUMN_NAME = 'Finding Label'

In [33]:
# --- 2. 데이터 로드 및 경로/레이블 준비 ---
print("데이터를 로드하고 파일 경로를 준비합니다...")
df = pd.read_csv(csv_path)
image_paths = (Path(image_dir) / df[ACTUAL_IMAGE_COLUMN_NAME]).astype(str).tolist()
labels = df[ACTUAL_LABEL_COLUMN_NAME].tolist()
print(f"총 {len(image_paths)}개의 이미지 데이터를 준비했습니다.")

데이터를 로드하고 파일 경로를 준비합니다...
총 984개의 이미지 데이터를 준비했습니다.


In [34]:
# --- 3. BioViL-T 모델 및 임베딩 함수 준비 ---
print("BioViL-T 이미지 모델을 로딩합니다...")
device = "cuda" if torch.cuda.is_available() else "cpu"
# ✅ [최종-진짜] 모델과 변환기를 각각의 변수에 저장합니다.
image_model = get_biovil_t_image_encoder()
image_transform = create_chest_xray_transform_for_inference(resize=512, center_crop_size=480)

image_model.to(device) # 모델을 GPU/CPU로 이동


BioViL-T 이미지 모델을 로딩합니다...


ImageModel(
  (encoder): MultiImageEncoder(
    (encoder): ResNetHIML(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (rel

In [35]:
# --- 4. 전체 이미지에 대한 임베딩 생성 루프 ---
print("\n전체 이미지에 대한 임베딩 생성을 시작합니다... (tqdm 진행률 표시줄 확인)")
embeddings_list = []
valid_labels = []

for path, label in tqdm(zip(image_paths, labels), total=len(image_paths)):
    embedding = get_image_embedding(path, image_model, image_transform)
    if embedding is not None:
        embeddings_list.append(embedding)
        valid_labels.append(label)


전체 이미지에 대한 임베딩 생성을 시작합니다... (tqdm 진행률 표시줄 확인)


100%|██████████| 984/984 [01:57<00:00,  8.39it/s]


In [36]:
# --- 5. 최종 결과 저장 ---
X_embeddings = np.array(embeddings_list)
y_labels = np.array(valid_labels)

np.save('cxr_embeddings.npy', X_embeddings)
np.save('cxr_labels.npy', y_labels)

print(f"\n✅ 임베딩 생성 및 저장이 완료되었습니다!")
print(f"생성된 임베딩의 Shape: {X_embeddings.shape}")
print(f"처리된 레이블의 개수: {len(y_labels)}")


✅ 임베딩 생성 및 저장이 완료되었습니다!
생성된 임베딩의 Shape: (984, 1, 512)
처리된 레이블의 개수: 984


In [ ]:
# ✅ 이 부분을 환경에 맞게 수정해주세요.
my_image_path = "/Users/solkim/Desktop/projects/medical/project/data/chestxray14/bbox_images/00030323_028.png"  # 예시: "data/my_cxr.jpg"
my_text_query = "Findings consistent with pneumonia"

# 위에서 정의한 함수를 호출하여 점수를 계산합니다.
final_score = calculate_image_text_similarity_official(my_image_path, my_text_query)

# 최종 결과를 출력합니다.
print(f"\n--- Result ---")
print(f"Image Path: '{my_image_path}'")
print(f"Text Query: '{my_text_query}'")
print(f"Final Similarity Score: {final_score:.4f}")